# Exercício de Regressão: Predição de Ruído de Aeroflóquio

## Objetivo
Prever o nível de ruído (em decibéis) gerado por aeroflóquios com base em características aerodinâmicas, utilizando diferentes modelos de machine learning.

## Base de Dados: `airfoil_self_noise`
Dataset disponível no OpenML com medições experimentais de ruído em aeroflóquios da NASA.

### Dicionário de Atributos (Traduzido)
| Nome Original       | Descrição em Português                     | Unidade/Tipo   |
|---------------------|-------------------------------------------|---------------|
| Frequency           | Frequência do som                         | Hertz (Hz)    |
| Angle_of_Attack     | Ângulo de ataque da asa                   | Graus (°)     |
| Chord_Length        | Comprimento da corda aerodinâmica         | Metros (m)    |
| Free_Velocity       | Velocidade do fluxo de ar livre           | Metros/segundo (m/s) |
| Suction_Side        | Espessura da camada de sucção/displacamento | Metros (m)    |
| Scaled_Sound        | Nível de pressão sonora escalonado (target)| Decibéis (dB) |

In [2]:
airfoil = fetch_openml(name='airfoil_self_noise', version=1, as_frame=True, parser='pandas')
X = airfoil.data
y = airfoil.target.astype(float)